In [2]:
import pandas as pd
import json
import os
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F

In [2]:
train = pd.read_json("./../final_data/L1_species_based_train_data.json", orient="records")
val = pd.read_json("./../final_data/L1_species_based_val_data.json", orient="records")
test = pd.read_json("./../final_data/L1_species_based_test_data.json", orient="records")

In [7]:
for level in ["L1", "L2"]:
    for base in ["species", "spatial"]:
        for split in ["train", "test", "val"]:
            df = pd.read_json(f"./../final_data/{level}_{base}_based_{split}_data.json", orient="records").drop("zone_id", axis=1)
            changed_cols = ['set_based_class', 'species_key']
            """for col in df.columns:
                if isinstance(df[col][0],list):
                    changed_cols.append(col)"""
            for col in changed_cols:
                df[col] = df[col].apply(lambda x : json.dumps(x))
            print(f"{level}_{base}_{split}")
            print(len(df[changed_cols]))
            print(len(df[changed_cols].drop_duplicates()))

L1_species_train
18130
11429
L1_species_test
19052
12250
L1_species_val
1014
1013
L1_spatial_train
15504
10332
L1_spatial_test
7934
5386
L1_spatial_val
2525
1979
L2_species_train
18155
12069
L2_species_test
19047
13097
L2_species_val
1025
1021
L2_spatial_train
15504
10331
L2_spatial_test
7934
5387
L2_spatial_val
2525
1979


In [17]:
df = pd.read_json(f"./../final_data/L1_spatial_based_train_data.json", orient="records").drop(["zone_id","shape_area","maps_based_class"], axis=1)
print(df.head())
print(len(df))
changed_cols = ['set_based_class',"species_based_class","species_key"]
for col in changed_cols:
    df[col] = df[col].apply(lambda x : json.dumps(x))
print(len(df.drop_duplicates()))
df = df[changed_cols].drop_duplicates()
print(len(df))
a = df.groupby("species_key")["set_based_class"].nunique()
b = df.groupby("species_key")["species_based_class"].nunique()

print(len(df.loc[~df["species_key"].isin(a[pd.DataFrame(a)["set_based_class"]>1].index)]))
print(len(df.loc[~df["species_key"].isin(a[pd.DataFrame(b)["species_based_class"]>1].index)]))

   split                           species_key  species_count  \
0  train  [3032837, 3170807, 3105433, 2883073]              4   
1  train                             [4299368]              1   
2  train                             [4299368]              1   
3  train                             [4299368]              1   
4  train                             [4299368]              1   

           species_based_class  species_based_num_classes  \
0  [0, 0, 1, 1, 1, 1, 0, 0, 0]                          4   
1  [0, 0, 1, 1, 1, 1, 1, 1, 1]                          7   
2  [0, 0, 1, 1, 1, 1, 1, 1, 1]                          7   
3  [0, 0, 1, 1, 1, 1, 1, 1, 1]                          7   
4  [0, 0, 1, 1, 1, 1, 1, 1, 1]                          7   

               set_based_class  set_based_num_classes  
0  [0, 0, 0, 1, 0, 0, 0, 0, 0]                      1  
1  [0, 0, 1, 1, 0, 0, 0, 1, 1]                      4  
2  [0, 0, 1, 1, 0, 0, 0, 1, 1]                      4  
3  [0, 0, 1, 1, 0,

: 

In [60]:
a.describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])

count    9917.000000
mean        1.136533
std         0.762891
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
90%         1.000000
95%         2.000000
99%         5.000000
max        23.000000
Name: set_based_class, dtype: float64

In [65]:
a[pd.DataFrame(a)["set_based_class"]>1]

species_key
[1035194]             2
[1340344]             2
[1427915]             2
[1540302]             2
[1540875]             2
                     ..
[9409796, 5228676]    2
[9409796]             4
[9616058]             4
[9701857]             2
[9761484]             2
Name: set_based_class, Length: 571, dtype: int64

In [61]:
b.describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])

count    9917.000000
mean        1.134214
std         0.918325
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
90%         1.000000
95%         2.000000
99%         5.000000
max        34.000000
Name: species_based_class, dtype: float64

In [5]:
df = pd.read_json(f"./../final_data/L1_species_based_val_data.json", orient="records")
df.head()

,zone_id,maps_based_class,split,shape_area,species_based_class,species_based_num_classes,set_based_class,set_based_num_classes,species_key,species_count
0,91166,8,test,290337.806386,"[0, 0, 0, 1, 0, 1, 0, 0, 0]",2,"[0, 0, 0, 0, 0, 0, 0, 1, 0]",1,"[3033665, 7705930]",2
1,56494,9,train,21132.179454,"[0, 0, 1, 1, 0, 1, 1, 1, 0]",5,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[5409933, 2977234, 2927078, 9151957]",4
2,27519,8,test,374375.466699,"[0, 0, 0, 1, 0, 1, 0, 1, 0]",3,"[0, 0, 0, 0, 0, 0, 0, 1, 0]",1,"[3109086, 5420901]",2
3,58872,4,test,136153.026931,"[0, 0, 1, 1, 0, 1, 1, 0, 1]",5,"[0, 0, 0, 1, 0, 0, 0, 0, 0]",1,"[3050364, 3034825, 5409933, 5386897]",4
4,87966,6,train,45324.722328,"[0, 0, 0, 1, 0, 1, 0, 1, 0]",3,"[0, 0, 0, 0, 0, 1, 0, 0, 0]",1,"[3190653, 3169131]",2


In [6]:
df = pd.read_json(f"./../final_data/L1_spatial_based_val_data.json", orient="records")
df.head()

,zone_id,maps_based_class,split,shape_area,species_key,species_count,species_based_class,species_based_num_classes,set_based_class,set_based_num_classes
0,13719,7,val,2178.604385,[2891147],1,"[0, 0, 1, 1, 1, 1, 1, 1, 1]",7,"[1, 0, 1, 1, 1, 1, 1, 0, 1]",7
1,37198,4,val,13361.952990,[2891147],1,"[0, 0, 1, 1, 1, 1, 1, 1, 1]",7,"[1, 0, 1, 1, 1, 1, 1, 0, 1]",7
2,37255,9,val,2154.862325,[2891147],1,"[0, 0, 1, 1, 1, 1, 1, 1, 1]",7,"[1, 0, 1, 1, 1, 1, 1, 0, 1]",7
3,44175,3,val,97522.688415,[2891147],1,"[0, 0, 1, 1, 1, 1, 1, 1, 1]",7,"[1, 0, 1, 1, 1, 1, 1, 0, 1]",7
4,61844,4,val,120895.741425,[2891147],1,"[0, 0, 1, 1, 1, 1, 1, 1, 1]",7,"[1, 0, 1, 1, 1, 1, 1, 0, 1]",7


In [26]:
df = pd.read_json(f"./../final_data/L1_species_based_val_data.json", orient="records").drop("zone_id", axis=1)
changed_cols = ['set_based_class',"species_based_class","species_key"]
for col in changed_cols:
    df[col] = df[col].apply(lambda x : json.dumps(x))
a = df[changed_cols].drop_duplicates()
b = df[['species_based_class',"species_key"]].drop_duplicates()
df.loc[df["species_key"]==a.drop(b.index).species_key.values[0]]

,maps_based_class,split,shape_area,species_based_class,species_based_num_classes,set_based_class,set_based_num_classes,species_key,species_count
339,9,train,15934.053241,"[0, 0, 1, 1, 0, 1, 0, 0, 0]",3,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[2482613, 8429926]",2
762,4,train,58703.823181,"[0, 0, 1, 1, 0, 1, 0, 0, 0]",3,"[0, 0, 0, 1, 0, 0, 0, 0, 0]",1,"[2482613, 8429926]",2


In [28]:
df2 = pd.read_json(f"./../processed_data/temp_class.json", orient="records")

In [31]:
a.drop(b.index).species_key.values[0]

'[2482613, 8429926]'

In [32]:
df2.head()

,species_key,maps_based_class,set_based_num_classes,set_based_class
0,"[10160919, 2753146, 1949715, 7934873, 4299368]",[6],1,"[0, 0, 0, 0, 0, 1, 0, 0, 0]"
1,"[10160919, 3085450, 4408796, 1920237, 2480389,...",[3],1,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"
2,"[10160919, 5415065, 7270427, 2812375]",[4],1,"[0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,[10196288],[3],1,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"
4,"[1035194, 1096909]",[4],1,"[0, 0, 0, 1, 0, 0, 0, 0, 0]"


In [34]:
df2.loc[df2["species_key"]=="[10196288]"]

,species_key,maps_based_class,set_based_num_classes,set_based_class
3,[10196288],[3],1,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [30]:
df2.loc[df2["species_key"]==a.drop(b.index).species_key.values[0]]

,species_key,maps_based_class,set_based_num_classes,set_based_class


In [45]:
train["species_key"] = train["species_key"].apply(lambda x : json.dumps(x))
val["species_key"] = val["species_key"].apply(lambda x : json.dumps(x))
train_sets = train["species_key"].to_list()
val_sets = val["species_key"].to_list()

In [46]:
redundant = [a for a in val_sets if a in train_sets]
val_sets_reduced = [a for a in val_sets if a not in redundant]
len(redundant)

983

In [47]:
temp = val
len(temp)

2029

In [50]:
temp[~temp["species_key"].isin(redundant)]

,zone_id,maps_based_class,split,shape_area,species_based_class,species_based_num_classes,set_based_class,set_based_num_classes,species_key,species_count
0,82763,6,train,54787.006275,"[0, 0, 0, 1, 0, 1, 0, 1, 0]",3,"[0, 0, 0, 0, 0, 1, 0, 0, 0]",1,"[2769766, 2650669]",2
1,18129,4,test,96501.944773,"[0, 0, 1, 1, 0, 1, 0, 1, 0]",4,"[0, 0, 0, 1, 0, 0, 0, 0, 0]",1,"[2888615, 8262702, 3173338, 8324121, 4201709]",5
2,6249,4,train,23662.198487,"[0, 0, 1, 1, 0, 1, 1, 0, 1]",5,"[0, 0, 0, 1, 0, 0, 0, 0, 0]",1,"[5229493, 5228676, 2490719, 2482553]",4
3,77494,9,val,5434.243579,"[0, 0, 0, 1, 0, 1, 0, 0, 0]",2,"[0, 0, 0, 0, 0, 0, 0, 0, 1]",1,"[2927298, 5667864, 3140574, 8176911, 5377429, ...",7
7,22309,4,train,154802.294684,"[0, 0, 0, 1, 0, 1, 1, 1, 1]",5,"[0, 0, 0, 1, 0, 0, 0, 0, 0]",1,"[2494686, 3039203]",2
...,...,...,...,...,...,...,...,...,...,...
2019,35336,6,test,201374.426662,"[0, 0, 1, 1, 0, 1, 0, 0, 0]",3,"[0, 0, 0, 0, 0, 1, 0, 0, 0]",1,"[5098489, 1685554, 8196495, 1703827, 1708512, ...",6
2021,18655,4,train,98282.994987,"[0, 0, 1, 1, 0, 1, 0, 0, 0]",3,"[0, 0, 0, 1, 0, 0, 0, 0, 0]",1,"[8108066, 5411475, 9151957, 2927007]",4
2024,9081,4,train,152011.590880,"[0, 0, 1, 1, 0, 1, 0, 1, 0]",4,"[0, 0, 0, 1, 0, 0, 0, 0, 0]",1,"[6097924, 4472897]",2
2025,39584,6,test,165907.627967,"[0, 1, 1, 1, 1, 1, 0, 1, 0]",6,"[0, 0, 0, 0, 0, 1, 0, 0, 0]",1,[1795215],1


In [40]:
temp["species_key"]

0                                    "[2769766, 2650669]"
1         "[2888615, 8262702, 3173338, 8324121, 4201709]"
2                  "[5229493, 5228676, 2490719, 2482553]"
3       "[2927298, 5667864, 3140574, 8176911, 5377429,...
4                                    "[7923515, 3033289]"
                              ...                        
2024                                 "[6097924, 4472897]"
2025                                          "[1795215]"
2026                                          "[2490255]"
2027               "[3063016, 5357013, 5371818, 8211070]"
2028                                 "[2889173, 3173338]"
Name: species_key, Length: 2029, dtype: object

In [25]:
a = train["species_key"]
b = val["species_key"]
df = pd.concat([a,b])

In [26]:
df.value_counts()

[3173338]                                                 653
[2808565]                                                 328
[5389029]                                                 308
[2889173]                                                 272
[5352251]                                                 227
                                                         ... 
[2437377, 4201709]                                          1
[7270427, 7791558, 1096909, 4452079, 4535636]               1
[4532544, 1067409, 7410701, 4299597, 4428942]               1
[8167638, 1919330, 5139156, 5137582, 1920218, 5411475]      1
[3063016, 5357013, 5371818, 8211070]                        1
Name: species_key, Length: 11158, dtype: int64

In [22]:
def get_onehots(entry, unique_classes):
    """Retrieve one_hot encodings"""
    x = [unique_classes.index(c) for c in entry]
    return F.one_hot(torch.tensor(x), num_classes=len(unique_classes)).sum(dim=0).tolist()

def get_species_classes(records, unique_classes, level, filter_uncommon=False):
    """Get one_hot encoded classes for each species"""
    if filter_uncommon:
        species_classes_counts = pd.DataFrame(records[["species_key",level]].value_counts()).reset_index().rename(columns={0:"count"})
        species_classes_counts = species_classes_counts.join(pd.DataFrame(records["species_key"].value_counts()).rename(columns={"species_key":"total"})["total"], on="species_key", how="inner")
        species_classes_counts["fraction"] = species_classes_counts["count"]/species_classes_counts["total"]
        #for sk in species_classes_counts["species_key"].unique():
        #    species_classes_counts.loc[species_classes_counts["species_key"]==sk,"cutoff"] = species_classes_counts.loc[species_classes_counts["species_key"] == sk,"fraction"].quantile(0.003)
        #records = species_classes_counts[species_classes_counts["fraction"]>species_classes_counts["cutoff"]]
        #records = species_classes_counts[species_classes_counts["fraction"]>species_classes_counts["cutoff"]]
        records = species_classes_counts[species_classes_counts["fraction"]>0.01]
    species_classes = records.groupby("species_key")[level].unique().reset_index().rename(columns={level:"classes"})
    species_classes["classes_onehot"] = species_classes["classes"].apply(lambda x : get_onehots(x, unique_classes))
    return species_classes

In [3]:
def check(species_list, reference_list):
    out = 1
    for i in species_list:
        if i not in reference_list:
            out = 0
    return out

In [4]:
def retrieve_in_ref_list(species_list, ref_list):
    out = [spe for spe in species_list if spe in ref_list]
    return out

In [5]:
a = pd.read_json("/data/nicola/WSH/final_data/L2_species_keys.json", orient="records").set_index("ID")
#a = temp.loc[a["species_key"].to_list()]
half1 = a.sample(frac=0.5, random_state=42)
half2 = a.drop(half1.index)

In [6]:
half2

,species_key
ID,
1,3170807
2,3105433
3,2883073
4,4299368
5,2891147
...,...
3612,8961574
3616,2680229
3618,2682592


In [8]:
all_data = pd.read_json("/data/nicola/WSH/final_data/L2_all_data.json", orient="records")
all_data["half1_species"] = all_data["species_key"].apply(lambda x : retrieve_in_ref_list(x,half1["species_key"].to_list()))
all_data["half2_species"] = all_data["species_key"].apply(lambda x : retrieve_in_ref_list(x, half2["species_key"].to_list()))

In [9]:
all_data["len_half1"] = all_data["half1_species"].apply(lambda x : len(x))
all_data["len_half2"] = all_data["half2_species"].apply(lambda x : len(x))

In [10]:
all_data[["zone_id","species_key","half1_species","half2_species","species_based_class"]]

,zone_id,species_key,half1_species,half2_species,species_based_class
0,9,"[3032837, 3170807, 3105433, 2883073]",[3032837],"[3170807, 3105433, 2883073]","[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, ..."
1,10,[4299368],[],[4299368],"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, ..."
2,13,[2891147],[],[2891147],"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, ..."
3,26,"[5137582, 7799370]","[5137582, 7799370]",[],"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, ..."
4,29,"[8207244, 5352367, 3170807]",[],"[8207244, 5352367, 3170807]","[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, ..."
...,...,...,...,...,...
25958,98095,"[3170040, 2812375, 3029627, 1537719, 3189747, ...","[3170040, 2812375, 3029627, 1537719, 3189747, ...",[8890062],"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, ..."
25959,98096,[3928139],[3928139],[],"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, ..."
25960,98097,[2882835],[],[2882835],"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, ..."
25961,98101,"[5275365, 3112620]",[5275365],[3112620],"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, ..."


In [11]:
split_1 = all_data.loc[all_data["len_half1"]>0].drop(["len_half2","half2_species","species_key","species_count"], axis=1).rename(columns={"half1_species":"species_key","len_half1":"species_count"})
split_2 = all_data.loc[all_data["len_half2"]>0].drop(["len_half1","half1_species","species_key","species_count"], axis=1).rename(columns={"half2_species":"species_key","len_half2":"species_count"})

In [15]:
split_1

,zone_id,maps_based_class,split,shape_area,species_based_class,num_classes,species_key,species_count
0,9,45,train,665918.930058,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, ...",9,[3032837],1
3,26,43,train,400775.100939,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, ...",9,"[5137582, 7799370]",2
7,61,43,train,46156.868586,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, ...",5,"[7270427, 8152305, 2482553]",3
8,62,43,train,267678.647708,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, ...",2,[5769191],1
10,66,43,train,70517.961898,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, ...",11,[5140244],1
...,...,...,...,...,...,...,...,...
25956,98092,53,test,4501.734903,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, ...",14,[3033289],1
25958,98095,43,test,63666.903126,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, ...",6,"[3170040, 2812375, 3029627, 1537719, 3189747, ...",6
25959,98096,53,test,414.649751,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, ...",10,[3928139],1
25961,98101,66,test,52057.227924,"[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, ...",9,[5275365],1


In [13]:
split_1.describe()

,zone_id,maps_based_class,shape_area,num_classes,species_count
count,19955.000000,19955.000000,1.995500e+04,19955.000000,19955.000000
mean,48538.779604,50.682335,1.539950e+05,8.907793,2.460937
std,27893.566574,24.056461,5.698747e+05,4.905340,1.789828
min,9.000000,2.000000,1.794476e+01,0.000000,1.000000
25%,23763.000000,42.000000,1.561038e+04,5.000000,1.000000
50%,49330.000000,45.000000,5.457341e+04,9.000000,2.000000
75%,71173.000000,66.000000,1.476097e+05,12.000000,4.000000
max,98118.000000,94.000000,3.100713e+07,21.000000,10.000000


In [14]:
split_1.len_half1.value_counts()

AttributeError: 'DataFrame' object has no attribute 'len_half1'

In [14]:
split_2.describe()

,zone_id,maps_based_class,shape_area,species_count,num_classes,len_half2
count,19310.000000,19310.000000,1.931000e+04,19310.000000,19310.000000,19310.000000
mean,48965.372294,50.755567,1.556372e+05,4.592595,8.572553,2.516054
std,27570.263443,23.850685,5.522028e+05,3.407622,4.615418,1.832243
min,9.000000,2.000000,8.114492e+00,1.000000,0.000000,1.000000
25%,24607.000000,42.000000,1.729896e+04,2.000000,5.000000,1.000000
50%,49507.500000,45.000000,5.762347e+04,3.000000,8.000000,2.000000
75%,71234.000000,66.000000,1.521167e+05,8.000000,12.000000,4.000000
max,98101.000000,94.000000,3.100713e+07,10.000000,21.000000,10.000000


In [15]:
split_2.len_half2.value_counts()

1     8485
2     3553
3     2238
4     1745
5     1463
6     1062
7      539
8      189
9       32
10       4
Name: len_half2, dtype: int64

In [16]:
train = pd.read_json("/data/nicola/WSH/final_data/L2_train_data.json", orient="records")
train_species = train["species_key"]
train_species = train_species.to_list()
train_species = sum(train_species, [])
train_species = list(set(train_species))
len(train_species)

3137

In [17]:
val = pd.read_json("/data/nicola/WSH/final_data/L2_val_data.json", orient="records")
val_species = val["species_key"]
val_species = val_species.to_list()
val_species = sum(val_species, [])
val_species = list(set(val_species))
len(val_species)

1625

In [18]:
test = pd.read_json("/data/nicola/WSH/final_data/L2_test_data.json", orient="records")
test_species = test["species_key"]
test_species = test_species.to_list()
test_species = sum(test_species, [])
test_species = list(set(test_species))
len(test_species)

2470

In [19]:
common_train_test_species = [spe for spe in test_species if spe in train_species]

In [20]:
len(common_train_test_species)

2077

In [21]:
all = pd.read_json("/data/nicola/WSH/final_data/L2_all_data.json", orient="records")
all["species_key"] = all["species_key"].apply(lambda x : retrieve_in_ref_list(x,common_train_test_species))
all["species_count"] = all["species_key"].apply(lambda x : len(x))
all = all[all["species_count"]>0]
all.describe()

,zone_id,maps_based_class,shape_area,species_count,num_classes
count,25753.000000,25753.000000,2.575300e+04,25753.000000,25753.000000
mean,48666.938764,50.615695,1.387388e+05,3.670252,9.844989
std,27949.581840,23.933494,5.126684e+05,3.173786,4.921894
min,9.000000,2.000000,8.114492e+00,1.000000,0.000000
25%,24009.000000,42.000000,1.294584e+04,1.000000,6.000000
50%,49354.000000,45.000000,4.671261e+04,2.000000,10.000000
75%,71432.000000,66.000000,1.325505e+05,6.000000,14.000000
max,98118.000000,94.000000,3.100713e+07,10.000000,21.000000
